In [4]:
# base imports
import pandas as pd
import numpy as np

In [5]:
# Data preparation: 
# Some data is missing (but we know what it should be) or some data should be changed.
df = pd.read_csv('./speed_dating_data.csv', encoding = 'ISO-8859-1')

def fooCareer(col):
    selected_df = df[col]
    career_c = {
        'lawyer': 1.,
        'law': 1.,
        'Economics': 7.,
        'tech professional': 5.
    }
    
    for x, row in selected_df.iterrows():
            career = df.at[x, col[0]]
            if career in career_c:
                df.at[x, col[1]] = career_c[career]
                

def fooGoal(col):
    selected_df = df[col]
    goal_c = {
        1: 1,
        2: 1,
        3: 1,
        4: 0,
        5: 1,
        6: 1,
    }
    
    for x, row in selected_df.iteritems():
        goal = df.at[x, col]
        if goal in goal_c:
            df.at[x, col] = goal_c[goal]

fooCareer(['career','career_c'])
fooGoal('goal')
df.to_csv('base_dating_data.csv', index=False)

In [6]:
# data is inconsistent:
# Some values (it’s based on waves) are 1-10 and others are 1-100 so
# they should be normalized so that all are 1-100.
df = pd.read_csv('base_dating_data.csv')

def func(lista):
    selected_df = df[lista]
    for x, row in selected_df.iterrows():
        multiplier = 100/row.sum()
        for col in lista:
            df.at[x, col] *= multiplier

lista = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
lista1 = ['attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o']
lista3 = ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
lista4 = ['attr1_2', 'sinc1_2', 'intel1_2', 'fun1_2', 'amb1_2', 'shar1_2']

func(lista)
func(lista1)
func(lista3)
func(lista4)

df.to_csv('base_dating_data.csv', index=False)

<ipython-input-6-023d9bb5a582>:9: RuntimeWarning: divide by zero encountered in double_scalars
  multiplier = 100/row.sum()
<ipython-input-6-023d9bb5a582>:11: RuntimeWarning: invalid value encountered in double_scalars
  df.at[x, col] *= multiplier


In [7]:
# Column met_o had values 1/2. It needs to be changed to 0/1.
df = pd.read_csv('./base_dating_data.csv')
df['met_o'] = df['met_o'].apply(lambda x: x - 1)
df.to_csv('base_dating_data.csv', index=False)

We removed some rows (by hand) because they didn't have `pid`.

### Below you can see code that prepares data for the basic matrix factorization.
Here in the base matrix we only have information about match. So we need a set of vectors, where each vector describes each date (holds both ids and information about match).

Task will requite two base matrices.
1. Matrix where men are "users" and women are "products". It will then be used to recommend women to men because matrix will say what's the predicted rating of a woman in eyes of man. Basically it will answer the question: **"How likely is that a man will like a woman?"**. Let's call this matrix/data frame **"men_like_women"**.
1. Matrix where women are "users" and men are "products". It will then be used to recommend men to women because matrix will say what's the predicted rating of a man in eyes of woman. Basically it will answer the question: **"How likely is that a woman will like a man?"**. Let's call this matrix/data frame **"women_like_men"**.

Why such analogies? It may help to understand how do this human relations task translates into recommender systems world.

These matrices will be used to train models (with matrix factorization) which will then be saved into csv files.

In [8]:
# Split into vectors, let's have two matrices as describes above.
base_df = pd.read_csv('./data_for_matrix.csv')
men_like_women_data = []
women_like_men_data = []

for _, row in base_df.iterrows():
    vector = {
        'id': row['iid'],
        'pid': row['pid'],
        'decision': row['dec'],
    }
    if row['gender'] == 0:
        # it's a woman
        women_like_men_data.append(vector)
    else:
        men_like_women_data.append(vector)

men_like_women_df = pd.DataFrame(men_like_women_data)
women_like_men_df = pd.DataFrame(women_like_men_data)

print('\n\nmen_like_women_df:')
print(men_like_women_df)
print('\nwomen_like_men_df:')
print(women_like_men_df)



men_like_women_df:
         id    pid  decision
0      11.0    1.0       0.0
1      11.0    2.0       0.0
2      11.0    3.0       0.0
3      11.0    4.0       0.0
4      11.0    5.0       0.0
...     ...    ...       ...
4179  552.0  526.0       0.0
4180  552.0  527.0       0.0
4181  552.0  528.0       0.0
4182  552.0  529.0       0.0
4183  552.0  530.0       0.0

[4184 rows x 3 columns]

women_like_men_df:
         id    pid  decision
0       1.0   11.0       1.0
1       1.0   12.0       1.0
2       1.0   13.0       1.0
3       1.0   14.0       1.0
4       1.0   15.0       1.0
...     ...    ...       ...
4179  530.0  548.0       0.0
4180  530.0  549.0       1.0
4181  530.0  550.0       0.0
4182  530.0  551.0       0.0
4183  530.0  552.0       1.0

[4184 rows x 3 columns]


## Let's prepare train and test data sets

### Make ids contiguous
We need to change the data. As you can see ids are not contiguous and they must be. So let's change that.

In [9]:
def proc_col(column):
    """Encodes a pandas column with continous ids."""
    unique = column.unique()
    old_to_new = {o: i for i, o in enumerate(unique)}
    return np.array([old_to_new.get(x, -1) for x in column])


def encode_data(df):
    """Encodes data with continous person and partner ids."""
    df = df.copy()
    for col_name in ["id", "pid"]:
        col = proc_col(df[col_name])
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

And now use these functions to make ids contiguous.

In [10]:
print('Before:')
print(men_like_women_df)
men_like_women_df = encode_data(men_like_women_df)
print('\nAfter:')
print(men_like_women_df)

women_like_men_df = encode_data(women_like_men_df)

Before:
         id    pid  decision
0      11.0    1.0       0.0
1      11.0    2.0       0.0
2      11.0    3.0       0.0
3      11.0    4.0       0.0
4      11.0    5.0       0.0
...     ...    ...       ...
4179  552.0  526.0       0.0
4180  552.0  527.0       0.0
4181  552.0  528.0       0.0
4182  552.0  529.0       0.0
4183  552.0  530.0       0.0

[4184 rows x 3 columns]

After:
       id  pid  decision
0       0    0       0.0
1       0    1       0.0
2       0    2       0.0
3       0    3       0.0
4       0    4       0.0
...   ...  ...       ...
4179  276  269       0.0
4180  276  270       0.0
4181  276  271       0.0
4182  276  272       0.0
4183  276  273       0.0

[4184 rows x 3 columns]


### Let's split into train and test data sets
Standard sklearn function `train_test_split` doesn't do a job here because both test and train data sets should include all people. So after using `train_test_split` we need to transition some people between sets to ensure that both sets are correct. The same goes for partners.

In [11]:
from sklearn.model_selection import train_test_split as tts

def train_test_split(df, test_size=0.2):
    # Use standard train_test_split.
    df_train, df_test = tts(df, test_size=test_size)
    no_train_unique = len(df_train.id.unique())
    no_test_unique = len(df_test.id.unique())
    
    # See what people are missing in the test set.
    diff = np.setdiff1d(df_train.id.unique(), df_test.id.unique())
    for id in diff:
        # For every missing person we need to exchange them for a
        # different one so that sets still have the same number of elements.
        person_to_send_to_test = df_train.loc[df_train.id == id].iloc[0]
        person_to_send_to_train = None
        # Choose some person from the test set to send to the train set.
        ids = df_test.id.unique()
        np.random.shuffle(ids)
        for test_id in ids:
            person = df_test.loc[df_test.id == test_id]
            if len(person.index) > 1:
                person_to_send_to_train = person.iloc[0]
                break
        if person_to_send_to_train is not None:
            # Remove people that transit from old sets.
            # .name holds the id of that row in the oryginal df.
            df_train = df_train.drop(person_to_send_to_test.name)
            df_test = df_test.drop(person_to_send_to_train.name)
            # Add new people to sets.
            df_train = pd.concat([df_train, person_to_send_to_train.to_frame().T], ignore_index=True)
            df_test = pd.concat([df_test, person_to_send_to_test.to_frame().T], ignore_index=True)
        else:
            raise Exception("Couldn't find any person from people to send from the test to the train.")
        
    # See what partners are missing in the test set.
    diff = np.setdiff1d(df_train.pid.unique(), df_test.pid.unique())
    for pid in diff:
        # For every missing partner we need to exchange them for a
        # different one so that sets still have the same number of elements.
        partner_to_send_to_test = df_train.loc[df_train.pid == pid].iloc[0]
        partner_to_send_to_train = None
        
        # Choose some partner from the test set to send to the train set.
        # Need to make sure that both sets will still have all the people.
        pids = df_test.pid.unique()
        np.random.shuffle(pids)
        for test_pid in pids:
            partner = df_test.loc[df_test.pid == test_pid]
            if len(partner.index) > 1:
                # Make sure we don't remove a person completely.
                id = partner.iloc[0].id
                person_qty = len(df_test.loc[df_test.id == id].index)
                if person_qty > 1:
                    partner_to_send_to_train = partner.iloc[0]
                    break
        if partner_to_send_to_train is not None:
            # Remove partners that transit from old sets.
            df_train = df_train.drop(partner_to_send_to_test.name)
            df_test = df_test.drop(partner_to_send_to_train.name)
            # Add new people to sets.
            df_train = pd.concat([df_train, partner_to_send_to_train.to_frame().T], ignore_index=True)
            df_test = pd.concat([df_test, partner_to_send_to_test.to_frame().T], ignore_index=True)
        else:
            raise Exception("Couldn't find any partner from partners to send from the test to the train.")
                    
    df_train = df_train.sort_values(by='id')
    df_test = df_test.sort_values(by='id')
    return df_train.reset_index(drop=True), df_test.sort_values(by='id').reset_index(drop=True)

### And now just get correct data

In [12]:
men_like_women_train_df, men_like_women_test_df = train_test_split(men_like_women_df, test_size=0.2)
men_like_women_no_men = len(men_like_women_train_df.id.unique())
men_like_women_no_women = len(men_like_women_train_df.pid.unique())

women_like_men_train_df, women_like_men_test_df = train_test_split(women_like_men_df, test_size=0.2)
women_like_men_no_women = len(women_like_men_train_df.id.unique())
women_like_men_no_men = len(women_like_men_train_df.pid.unique())

## Let's create matrix factorization models

We will create and train several base MF (matrix factorization) models and for each of them let's do a cross validation to learn the best hyperparameters and parameters. Then we will compare the results and choose the best model.

Some general explanations for models:
* Models are train on only one batch because our data set is rather small.

Good reading resource: https://towardsdatascience.com/weight-initialization-techniques-in-neural-networks-26c649eb3b78

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev

/home/bartosz/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

### Matrix factorization without bias

In [14]:
class MatrixFactorizationWithoutBiasNoXavier(nn.Module):
    def __init__(self, num_people, num_partners, weights=(0, 1), emb_size=100):
        super(MatrixFactorizationWithoutBiasNoXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        self.person_emb.weight.data.uniform_(weights[0], weights[1])
        self.partner_emb.weight.data.uniform_(weights[0], weights[1])
        
    def forward(self, u, v):
        u = self.person_emb(u)
        v = self.partner_emb(v)
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1))

    
class MatrixFactorizationWithoutBiasXavier(nn.Module):
    def __init__(self, num_people, num_partners, emb_size=100):
        super(MatrixFactorizationWithoutBiasXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        torch.nn.init.xavier_uniform_(self.person_emb.weight)
        torch.nn.init.xavier_uniform_(self.partner_emb.weight)
        
    def forward(self, u, v):
        u = self.person_emb(u)
        v = self.partner_emb(v)
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1))

    
# Example small models demonstrating weights
example_model_no_xavier = MatrixFactorizationWithoutBiasNoXavier(10, 10, emb_size=3)
example_model_xavier = MatrixFactorizationWithoutBiasXavier(10, 10, emb_size=3)
print("Model with without xavier weights are:\n")
for p in example_model_no_xavier.parameters():
    print(p)
print('\n\n', '='*20)
print("\n\nModel with with xavier weights are:\n")
for p in example_model_xavier.parameters():
    print(p)

Model with without xavier weights are:

Parameter containing:
tensor([[0.0065, 0.3367, 0.0883],
        [0.2679, 0.2831, 0.6683],
        [0.2382, 0.5243, 0.7572],
        [0.1230, 0.4315, 0.9713],
        [0.6620, 0.2248, 0.0148],
        [0.9528, 0.7787, 0.1875],
        [0.0267, 0.9823, 0.2607],
        [0.2889, 0.1464, 0.0992],
        [0.5508, 0.1048, 0.9712],
        [0.3939, 0.7173, 0.7019]], requires_grad=True)
Parameter containing:
tensor([[0.6970, 0.4621, 0.8775],
        [0.7556, 0.5646, 0.1766],
        [0.4267, 0.5393, 0.9096],
        [0.0856, 0.3851, 0.8909],
        [0.3469, 0.8165, 0.8905],
        [0.3670, 0.0513, 0.0907],
        [0.1287, 0.2685, 0.9530],
        [0.5219, 0.9746, 0.9852],
        [0.8413, 0.4131, 0.2198],
        [0.9447, 0.3644, 0.0070]], requires_grad=True)




Model with with xavier weights are:

Parameter containing:
tensor([[-0.2812, -0.5231, -0.6291],
        [-0.3628,  0.0871,  0.1083],
        [-0.3601,  0.4917, -0.6014],
        [-0.1387, -0

### Matrix factorization without bias

In [15]:
class MatrixFactorizationWithBiasNoXavier(nn.Module):
    def __init__(self, num_people, num_partners, weights=(0, 1), bias=(-0.01, 0.01), emb_size=100):
        super(MatrixFactorizationWithBiasNoXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.person_bias = nn.Embedding(num_people, 1)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        self.parnter_bias = nn.Embedding(num_partners, 1)
        self.person_emb.weight.data.uniform_(weights[0], weights[1])
        self.partner_emb.weight.data.uniform_(weights[0], weights[1])
        self.person_bias.weight.data.uniform_(bias[0], bias[1])
        self.parnter_bias.weight.data.uniform_(bias[0], bias[1])
            
    def forward(self, u, v):
        u = self.person_emb(u)
        v = self.partner_emb(v)
        bias_u = self.person_bias(u).squeeze()
        bias_v = self.parnter_bias(v).squeeze()
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1) + bias_u + bias_v)
    
    
class MatrixFactorizationWithBiasXavier(nn.Module):
    def __init__(self, num_people, num_partners, bias=(-0.01, 0.01), emb_size=100):
        super(MatrixFactorizationWithBiasXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.person_bias = nn.Embedding(num_people, 1)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        self.parnter_bias = nn.Embedding(num_partners, 1)
        torch.nn.init.xavier_uniform_(self.person_emb.weight)
        torch.nn.init.xavier_uniform_(self.partner_emb.weight)
        self.person_bias.weight.data.uniform_(bias[0], bias[1])
        self.parnter_bias.weight.data.uniform_(bias[0], bias[1])
            
    def forward(self, u, v):
        u = self.person_emb(u)
        v = self.partner_emb(v)
        bias_u = self.person_bias(u).squeeze()
        bias_v = self.parnter_bias(v).squeeze()
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1) + bias_u + bias_v)
    

# Example small models demonstrating weights
example_model_no_xavier = MatrixFactorizationWithBiasNoXavier(10, 10, bias=(0, 0), emb_size=3)
example_model_xavier = MatrixFactorizationWithBiasXavier(10, 10, emb_size=3)
print("Model with without xavier weights are:\n")
for p in example_model_no_xavier.parameters():
    print(p)
print('\n\n', '='*20)
print("\n\nModel with with xavier weights are:\n")
for p in example_model_xavier.parameters():
    print(p)

Model with without xavier weights are:

Parameter containing:
tensor([[0.3711, 0.9834, 0.2980],
        [0.9217, 0.1187, 0.9703],
        [0.4238, 0.8240, 0.8976],
        [0.8818, 0.9370, 0.1916],
        [0.5306, 0.5224, 0.0893],
        [0.6110, 0.1732, 0.1844],
        [0.6753, 0.1635, 0.4409],
        [0.2066, 0.3043, 0.7560],
        [0.6152, 0.8276, 0.2381],
        [0.8410, 0.7981, 0.8665]], requires_grad=True)
Parameter containing:
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], requires_grad=True)
Parameter containing:
tensor([[0.2940, 0.8486, 0.2826],
        [0.1572, 0.4870, 0.7651],
        [0.4908, 0.1995, 0.5992],
        [0.2625, 0.8239, 0.6352],
        [0.3506, 0.2808, 0.0287],
        [0.4312, 0.0188, 0.5292],
        [0.5918, 0.8088, 0.8146],
        [0.3070, 0.6349, 0.5282],
        [0.4303, 0.8334, 0.5562],
        [0.2999, 0.7426, 0.6360]], requires_grad=True)
Parameter c

### Training and testing functions are below

In [16]:
def test(model, df_test, verbose=False):
    model.eval()
    # .to(dev) puts code on either gpu or cpu.
    people = torch.LongTensor(df_test.id.values).to(dev)
    partners = torch.LongTensor(df_test.pid.values).to(dev)
    decision = torch.FloatTensor(df_test.decision.values).to(dev)
    y_hat = model(people, partners)
    loss = F.mse_loss(y_hat, decision)
    if verbose:
        print('test loss %.3f ' % loss.item())
    return loss.item()


# Default values assigned below are ones that I found online.
# Cross validadtion will be done later but it's good to have some defaults.
def train(model, df_train, epochs=100, learning_rate=0.01, weight_decay=1e-5, verbose=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    model.train()
    
    for epoch in range(epochs):
        # From numpy to PyTorch tensors.
        # .to(dev) puts code on either gpu or cpu.
        people = torch.LongTensor(df_train.id.values).to(dev)
        partners = torch.LongTensor(df_train.pid.values).to(dev)
        decision = torch.FloatTensor(df_train.decision.values).to(dev)
        
        # calls forward method of the model
        y_hat = model(people, partners)
        # Using mean squared errors loss function
        loss = F.mse_loss(y_hat, decision)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if verbose and epoch % 100 == 0: 
            print(loss.item())

# Let's finally train models and choose the best one

## Let's first train men_like_women

## Cross validation on each model

In [17]:
from itertools import product
from tqdm import tqdm

epochs_poss = list(range(10, 310, 10))
weight_decay_poss = [0.0, 1e-6, 1e-5, 1e-4, 0.0001, 0.001, 0.01, 0.1, 1.0]
learning_rate_poss = [1e-6, 1e-5, 1e-4, 0.0001, 0.001, 0.01, 0.1, 1.0]
weights_poss = [(0, 1), (-1, 1), (0, 0.2), (-0.2, 0.2), 'xavier']
bias_poss = [None, (0, 0), (-0.01, 0.01), (0, 1), (-1, 1)]
emb_size_poss = [10, 50, 100, 150]

def cross_vaild_model(model, train_df, test_df, verbose=False):
    """Function to choose the best hyperparameters for a model."""
    min_loss = float('inf')
    best_settings = None

    for (epochs, wd, lr) in tqdm(product(epochs_poss, weight_decay_poss, learning_rate_poss)):
        train(model, train_df, epochs=epochs, learning_rate=lr, weight_decay=wd)
        test_loss = test(model, test_df)
        if (test_loss < min_loss) or (test_loss == min_loss and best_settings is not None and epochs < best_settings['epochs']):
            min_loss = test_loss
            best_settings = {'epochs': epochs, 'weight_decay': wd, 'learning_rate_poss': lr}
    if verbose:
        print('min loss %.3f' % min_loss)
        print('best settings are', best_settings)
    return min_loss, best_settings


def cross_vaild(num_people, num_partners, train_df, test_df, verbose=False):
    """Function to choose the best model."""
    min_loss = float('inf')
    best_settings = None
    
    for (weights, bias, emb_size) in tqdm(product(weights_poss, bias_poss, emb_size_poss)):
        model = None
        if weights == 'xavier':
            if bias is None:
                model = MatrixFactorizationWithoutBiasXavier(num_people, num_partners, emb_size=emb_size).to(dev)
            else:
                model = MatrixFactorizationWithBiasXavier(num_people, num_partners, bias=bias, emb_size=emb_size).to(dev)
        else:
            if bias is None:
                model = MatrixFactorizationWithoutBiasNoXavier(num_people, num_partners, weights=weights, emb_size=emb_size).to(dev)
            else:
                model = MatrixFactorizationWithBiasNoXavier(num_people, num_partners, weights=weights, bias=bias, emb_size=emb_size).to(dev)
        
        model_min_loss, model_best_settings = cross_vaild_model(model, train_df, test_df, verbose)
        if (model_min_loss < min_loss) or (model_min_loss == min_loss and best_settings and emb_size < best_settings['emb_size']):
            min_loss = model_min_loss
            curr_settings = {'model': model, 'weights': weights, 'bias': bias, 'emb_size': emb_size}
            best_settings = {**curr_settings, **model_best_settings}
            
    return min_loss, best_settings

In [ ]:
# model.eval()
# print(model(torch.LongTensor(men_like_women_train_df.id).to(dev), torch.LongTensor(men_like_women_train_df.pid).to(dev)))

## Let's create logistic regression
Now, to create regression models, both used before and after, we will modify data set in such ways:
1. With modified income column - here the missing values are filled with the mean of the column and modified attr_5_1 columns - here the missing values are filled with the mean of the columns.
2. Without income and attr_5_1 columns.

#### Later we will aply the logic to combine this with MF.

It's going to be used:
1. Before matrix factorization - output of this model will be used as a input to MF.
2. Before matrix factorization - output of MF will be used as input in this model, as one of the features.

We will try 3 diffetent combinations:
1. Regression before MF.
2. Regression after MF.
3. Regression before and after MF.


### Data preprocessing for logistic regression

In [20]:
def fill_with_mean(df):
    """Function fills missing values with mean of the column"""
    col_means = df.mean()
    for index, row in df.iterrows():
        for column in df.columns:
            if column != 'field' and column != 'race_o' and column != 'met_o' and column != 'race' and column != 'goal' and column != 'date' and column != 'go_out' and column != 'career_c':
                if np.isnan(row[column]):
                    df.at[index, column] = col_means[column]
            elif column == 'field' and np.isnan(row['field']):
                df.at[index, column] = 8
            elif column == 'race_o' and np.isnan(row['race_o']):
                df.at[index, column] = 2
            elif column == 'race' and np.isnan(row['race']):
                df.at[index, column] = 2
            elif column == 'goal' and np.isnan(row['goal']):
                df.at[index, column] = 1
            elif column == 'date' and np.isnan(row['date']):
                df.at[index, column] = 6
            elif column == 'go_out' and np.isnan(row['go_out']):
                df.at[index, column] = 2
            elif column == 'career_c' and np.isnan(row['career_c']):
                df.at[index, column] = 2
            elif column == 'met_o' and (np.isnan(row['met_o']) or row['met_o'] != 0):
                df.at[index, column] = 1
    return df

### Option 1
Modified income column - here the missing values are filled with the mean of the column and modified attr_5_1 columns - here the missing values are filled with the mean of the columns.

In [18]:
# min_loss, best_settings = cross_vaild(men_like_women_no_men, men_like_women_no_women, men_like_women_train_df, men_like_women_test_df)
# print(min_loss)
# print(best_settings)
model = MatrixFactorizationWithoutBiasNoXavier(men_like_women_no_men, men_like_women_no_women).to(dev)
print(cross_vaild_model(model, men_like_women_train_df, men_like_women_test_df))

256it [01:36,  2.64it/s]


KeyboardInterrupt: 

In [93]:
import torch
from torch import Tensor

epochs_poss = list(range(10, 310, 10))
weight_decay_poss = [0.0, 1e-6, 1e-5, 1e-4, 0.0001, 0.001, 0.01, 0.1, 1.0]
learning_rate_poss = [1e-6, 1e-5, 1e-4, 0.0001, 0.001, 0.01, 0.1, 1.0]
weights_poss = [(0, 1), (-1, 1), (0, 0.2), (-0.2, 0.2), 'xavier']
bias_poss = [None, (0, 0), (-0.01, 0.01), (0, 1), (-1, 1)]
emb_size_poss = [10, 50, 100, 150]
lambda_param = 0.01
train_size = 0.9
test_size = 1 - train_size
learning_rate = 0.001

regression_df_1 = base_df.copy()
regression_df_1 = fill_with_mean(regression_df_1)
x_df = regression_df_1.copy()

x_df = x_df.drop(['dec'], axis=1).values
input_dim = len(regression_df_1.columns) - 1

y_df = regression_df_1['dec'].values  # 4

X = torch.tensor(x_df, dtype=torch.float32)  # 5
y = torch.tensor(y_df)

def one_hot_encode(vector):
    n_classes = len(vector.unique())  # 1
    one_hot = torch.zeros((vector.shape[0], n_classes))\
        .type(torch.LongTensor)  # 2
    return one_hot\
        .scatter(1, vector.type(torch.LongTensor).unsqueeze(1), 1)  # 3

y_one_hot = one_hot_encode(Y)

# Create train and test sets

random_indices = torch.randperm(X.shape[0])  # 1

n_train = int(train_size * X.shape[0])  # 2
X_train = X[random_indices[:n_train]]  # 3
y_train = y[random_indices[:n_train]]
y_train_one_hot = y_one_hot[random_indices[:n_train]]


X_test = X[random_indices[n_train:]]  # 4
y_test = y[random_indices[n_train:]]
y_test_one_hot = y_one_hot[random_indices[n_train:]]


w = torch.rand((input_dim, 2))
b = torch.rand(2)
Z = torch.mm(X, w) + b  # 1


def softmax_activation(z):
    exponentials = torch.exp(z)
    exponentials_row_sums = torch.sum(exponentials, axis=1).unsqueeze(1)  # 1
    return exponentials / exponentials_row_sums

def mse_loss(y_one_hot, activations):
    return -torch.mean(  # 3
        torch.sum(  # 2
            (y_one_hot - torch.log(activations))**2, axis=1  # 1
        )
    )





for i in range(1,300):
    Z = torch.mm(X_train, w) + b
    Z /= 50000
    A = softmax_activation(Z)
    l2_regularization = torch.sum(w ** 2)
    print(mse_loss(y_train_one_hot, A))
    print(lambda_param * l2_regularization)
    loss = mse_loss(y_train_one_hot, A) \
           + lambda_param * l2_regularization
    w_gradients = -torch\
        .mm(X_train.transpose(0, 1), y_train_one_hot - A) / n_train \
                  + (2 * lambda_param * w)
    b_gradients = -torch.mean(y_train_one_hot - A, axis=0)

    w -= learning_rate * w_gradients
    b -= learning_rate * b_gradients

    if i == 1 or i % 25 == 0:
        print("Loss at iteration {}: {}".format(i, loss))

test_predictions = torch.argmax(
    softmax_activation(torch.mm(X_test, w) + b), axis=1
)
test_accuracy = float(sum(test_predictions == y_test)) / y_test.shape[0]
print("\nFinal Test Accuracy: {}".format(test_accuracy))

tensor(-3.3593)
tensor(0.4764)
Loss at iteration 1: -2.8829398155212402
tensor(-65.8759)
tensor(0.8148)
tensor(-784.5463)
tensor(4.8255)
tensor(-449.9402)
tensor(2.9714)
tensor(-226.5029)
tensor(1.6856)
tensor(-1167.4216)
tensor(7.0544)
tensor(-6.0932)
tensor(0.4892)
tensor(-1024.4152)
tensor(6.2385)
tensor(-18.8773)
tensor(0.5539)
tensor(-1792.5759)
tensor(10.6267)
tensor(-50.9181)
tensor(0.7326)
tensor(-833.1152)
tensor(5.1002)
tensor(-414.4613)
tensor(2.7702)
tensor(-252.8977)
tensor(1.8322)
tensor(-1109.9382)
tensor(6.7261)
tensor(-9.9025)
tensor(0.5080)
tensor(-1492.6503)
tensor(8.9115)
tensor(-12.6702)
tensor(0.5264)
tensor(-794.1476)
tensor(4.8792)
tensor(-442.7243)
tensor(2.9299)
tensor(-231.6676)
tensor(1.7137)
tensor(-1155.8062)
tensor(6.9875)
tensor(-6.7247)
tensor(0.4918)
tensor(-1137.4312)
tensor(6.8826)
tensor(-7.8653)
tensor(0.4974)
Loss at iteration 25: -7.367877960205078
tensor(-1298.0530)
tensor(7.7995)
tensor(-3.4316)
tensor(0.4761)
tensor(-4.0059)
tensor(0.4783)
ten

tensor(-376.0273)
tensor(2.5508)
tensor(-284.5632)
tensor(2.0068)
tensor(-1046.5255)
tensor(6.3627)
tensor(-16.1635)
tensor(0.5378)
tensor(-1750.2841)
tensor(10.3829)
tensor(-44.0799)
tensor(0.6933)
tensor(-855.5986)
tensor(5.2253)
tensor(-398.8687)
tensor(2.6802)
tensor(-265.3349)
tensor(1.8996)
tensor(-1084.3601)
tensor(6.5787)
tensor(-12.1613)
tensor(0.5177)
tensor(-1625.1599)
tensor(9.6676)
tensor(-26.4854)
tensor(0.5985)

Final Test Accuracy: 0.43130227001194743


### Option 2
Without income and attr_5_1 columns.

In [ ]:
regression_df_2 = base_df.copy()
cols_to_drop = ['income', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1']
regression_df_2 = regression_df_2.drop(columns=cols_to_drop)
regression_df_2 = fill_with_mean(regression_df_2)